In [2]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [3]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 96 kB 3.2 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [4]:
(tr_img, tr_cls), (te_img, te_cls) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


데이터가 너무 많으니까 학습 데이터는 2000개만 사용

In [5]:
tr_img = tr_img[:2000].astype('float32') / 255.0
te_img = te_img[:1000].astype('float32') / 255.0

In [23]:
tr_cls = tr_cls[:2000]
te_cls = te_cls[:1000]

In [19]:
def model_builder(hp) :
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape = (28, 28)))

  hp_units = hp.Int('units', min_value = 32, max_value = 128, step = 16)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  hp_lr = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_lr),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['acc'])
  
  return model

#### Dense의 노드 수와 optimizer의 learning_rate를 탐색한다.

In [20]:
tuner = kt.Hyperband(model_builder, objective = 'val_acc',
                     max_epochs = 10,
                     factor = 3,
                     directory = '/content/gdrive/My Drive/tensorflow_core',
                     project_name = 'intro_kt')

In [21]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [24]:
tuner.search(tr_img, tr_cls, epochs = 10, validation_data = (te_img, te_cls), callbacks = [ClearTrainingOutput()])

best = tuner.get_best_hyperparameters(num_trials = 1)[0]

Trial 29 Complete [00h 00m 02s]
val_acc: 0.7710000276565552

Best val_acc So Far: 0.828000009059906
Total elapsed time: 00h 01m 10s
INFO:tensorflow:Oracle triggered exit


In [26]:
best_model = tuner.hypermodel.build(best)

best_model.fit(tr_img, tr_cls, epochs = 15, validation_data = (te_img, te_cls))

Epoch 1/15
63/63 [==============================] - 1s 5ms/step - loss: 0.9850 - acc: 0.6510 - val_loss: 0.6441 - val_acc: 0.7470
Epoch 2/15
63/63 [==============================] - 0s 3ms/step - loss: 0.5996 - acc: 0.7865 - val_loss: 0.6765 - val_acc: 0.7440
Epoch 3/15
63/63 [==============================] - 0s 3ms/step - loss: 0.4964 - acc: 0.8285 - val_loss: 0.5825 - val_acc: 0.7870
Epoch 4/15
63/63 [==============================] - 0s 2ms/step - loss: 0.4466 - acc: 0.8475 - val_loss: 0.6545 - val_acc: 0.7610
Epoch 5/15
63/63 [==============================] - 0s 4ms/step - loss: 0.4091 - acc: 0.8540 - val_loss: 0.5738 - val_acc: 0.8100
Epoch 6/15
63/63 [==============================] - 0s 3ms/step - loss: 0.4172 - acc: 0.8580 - val_loss: 0.6506 - val_acc: 0.7630
Epoch 7/15
63/63 [==============================] - 0s 3ms/step - loss: 0.4139 - acc: 0.8460 - val_loss: 0.6131 - val_acc: 0.7920
Epoch 8/15
63/63 [==============================] - 0s 3ms/step - loss: 0.3296 - acc: 0.89